In [1]:
import os
import json
import numpy as np
from pandas import DataFrame, read_csv
from typing import List
from transformers import BertModel, BertTokenizer, BertTokenizerFast
import torch
import torch.nn.functional as F


2023-03-04 14:52:31.478453: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 14:52:31.595165: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-04 14:52:31.597784: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/ikonkobo/anaconda3/lib/:/home/ikonkobo/anaconda3/lib/:/home/ikonkobo/anacond

In [2]:
os.chdir("../")
from src.data.request_data import getDataTF, getDataHF
from src.data.processing_data import StackedFormat

In [3]:
with open(r"./inputs_data/tokens.json") as file:
    input = json.load(file)

model_id = input["model_id"]
hf_token = input["hf_token"]

In [4]:
# url_dyda_da = input["url_dyda_da"]
# df_dyda_da = getDataHF(url_dyda_da)
# getDataTF("dyda_da", "test")

In [5]:
Contexts, Labels = StackedFormat("dyda_da", 5).get_contexts_labels()

In [13]:
Contexts[2][:2]

["ok , that shall be arranged . sun-set hotel . may i help you ? hold on , please . let me check it for you . yes , you're right . you will keep it for 3 days . well , now i want to change the date from 24th to 28th . yes , i have booked a room for 24th . it's a double room .",
 "i know . i know . i'm really sorry . i lost my bag . i'll call the lost and found office . it's ten after six.we ' re late.but dinner is at six thirty . i didn't think of it.thank you.and i do apologize for being late . i'm sorry i'm so late . i had a really bad day ."]

In [7]:
Labels

[tensor([[2., 3., 2., 3., 2.],
         [2., 2., 2., 3., 2.],
         [2., 1., 2., 0., 3.],
         ...,
         [1., 3., 2., 2., 0.],
         [2., 2., 3., 2., 3.],
         [0., 1., 3., 3., 2.]]),
 tensor([[3., 2., 3., 1., 1.],
         [2., 3., 3., 2., 2.],
         [3., 2., 2., 3., 2.],
         [2., 2., 3., 2., 3.],
         [2., 3., 2., 1., 3.],
         [3., 1., 1., 1., 2.],
         [1., 3., 1., 0., 1.],
         [1., 0., 1., 1., 0.],
         [0., 0., 1., 1., 2.],
         [2., 1., 3., 3., 3.],
         [2., 0., 1., 3., 3.],
         [2., 2., 2., 3., 3.],
         [1., 3., 2., 3., 0.],
         [2., 3., 1., 0., 1.],
         [0., 3., 2., 1., 1.],
         [2., 3., 3., 2., 3.],
         [2., 2., 3., 2., 3.],
         [2., 2., 2., 2., 2.],
         [3., 3., 2., 1., 2.],
         [3., 3., 2., 3., 1.],
         [3., 1., 2., 3., 2.],
         [0., 0., 1., 1., 1.],
         [1., 3., 3., 1., 2.],
         [3., 2., 2., 2., 3.],
         [3., 1., 2., 1., 1.],
         [3., 1., 0., 1

## Testing BERT embedding

In [8]:
model_name = 'bert-base-uncased'
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
"""
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

input_ids_tensor = torch.tensor(tokenizer.batch_encode_plus(Contexts,
                                     max_length=50,
                                     padding=True,
                                     truncation="only_first")["input_ids"][:3])

with torch.no_grad():
    embedds = model(input_ids_tensor)[0]
embedds
"""

'\ntokenizer = BertTokenizerFast.from_pretrained(\'bert-base-uncased\')\n\ninput_ids_tensor = torch.tensor(tokenizer.batch_encode_plus(Contexts,\n                                     max_length=50,\n                                     padding=True,\n                                     truncation="only_first")["input_ids"][:3])\n\nwith torch.no_grad():\n    embedds = model(input_ids_tensor)[0]\nembedds\n'

In [11]:
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = Contexts[1][:10]

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
def _reshape_(x: torch.Tensor): return x.reshape(x.shape[0], 1, x.shape[1])
sentence_embeddings = _reshape_(sentence_embeddings)

print("Sentence embeddings:")
print(sentence_embeddings)


Sentence embeddings:
tensor([[[-5.5844e-03,  2.9351e-02,  4.0798e-02,  ..., -3.0757e-03,
           2.3811e-02,  2.1139e-02]],

        [[ 2.0593e-02, -7.3245e-03,  5.1020e-02,  ..., -4.6680e-03,
           3.1919e-02,  1.2761e-02]],

        [[ 1.4871e-02,  3.0301e-03,  3.0222e-02,  ..., -3.6234e-02,
          -6.4907e-03,  5.1779e-03]],

        ...,

        [[-2.5901e-02,  1.6732e-02,  6.4316e-02,  ...,  5.8688e-03,
          -1.8238e-02,  2.6413e-02]],

        [[ 3.0525e-02, -6.6683e-03,  5.2210e-02,  ...,  3.3508e-02,
           1.1205e-02,  2.5979e-02]],

        [[ 2.3695e-02, -4.2341e-06,  3.2229e-02,  ..., -1.1047e-03,
           1.9255e-02,  1.8958e-02]]])


In [12]:
sentence_embeddings.shape

torch.Size([10, 1, 768])

In [ ]:
# model = BertModel.from_pretrained("bert-base-uncased")
# embedding_matrix = model.embeddings.word_embeddings.weight

In [ ]:
#def embedding(texts: List[str],
#              model_id: str,
#              hf_token: str):
#    api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
#    headers = {"Authorization": f"Bearer {hf_token}"}
#    response = requests.post(api_url, 
#                             headers=headers, 
#                             json={"inputs": texts, 
#                                   "options":{"wait_for_model":True}})
#    return response.json()